In [1]:
import pandas as pd
from dataworks import DataWorks
dws = DataWorks()



In [80]:
def load_choice_file(path, field, name):
    """读取choice数据终端导出的数据文件.
        Choiced导出文件格式对应的处理规则. 
        - 原文件第一行为“宏观数据“或类似内容,但read_excel方法未加载该行内容
        - 数据的第一行作为指标标题
        - 第一列作为日期
        - 前六行和最后一行都不是数据内容
        - 剔除“日期”字段为空的行和其他非数据内容（标识数据来源的文字内容）
        
        Choice文件导出注意事项:
        - 导出字段选择:指标名称、频率、单位、来源
        - 日期排序:降序
        - 图形设置:不导出图形
        - 勾选“使用函数方式导出”
    Args:
        file_path (str): Choice导出文件(.xlsx)的绝对路径

    Returns:
        dataframe: 将Choice导出文件内容加载到dataframe,并返回
    """
    if path=='':
        return None
    df = pd.read_excel(path)
    df.columns = df.iloc[0]
    df.rename(columns={df.columns[0]: 'date'}, inplace=True)
    df = df.iloc[6:]
    if isinstance(field, list):
        df[name] = df[field[0]] + df[field[1]]
        df = df[['date', name]]
    else:
        df.rename(columns={field: name}, inplace=True)
        df = df[['date', name]]
        
    # df.reset_index(drop=True, inplace=True)
    # df.set_index('date', inplace=True)
    df.dropna(axis=0, subset=['date'], inplace=True)
    df = df[df['date'] != '数据来源：东方财富Choice数据']
    df['date'] = pd.to_datetime(df['date'])
    return df

In [3]:
import akshare as ak
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date.iloc[0]

datetime.date(1990, 12, 19)

In [81]:
config = dws.get_variety_setting('setting/variety.json')
variety_config = config['RB']
combined_df = None
for data_type, data_info in variety_config['DataIndex'].items():
    # 从文件中加载指定字段的数据
    data_df = load_choice_file(data_info['Path'], data_info['Field'], data_info['Name'])
    
    # 将数据添加到combined_df中
    # combined_df = pd.concat([combined_df, data_df], axis=1)
    combined_df = pd.merge(combined_df, data_df, on='date', how='outer') if combined_df is not None else data_df
# combined_df.dropna(axis=0, subset=['date'], inplace=True)

In [2]:
import pandas as pd
df_basis = dws.get_data_by_sql('SELECT * FROM basis')
df_basis = df_basis[['date', 'dom_basis']]

In [3]:
df_basis['date'] = pd.to_datetime(df_basis['date'])
ts = df_basis.set_index('date')['dom_basis']

In [19]:
import re

def extract_variables(format_str):
    # 正则表达式模式，匹配非空字符（即变量）
    pattern = r'\{.*?\}'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(pattern, format_str)
    # 去除花括号并返回结果
    return [var[1:-1] for var in variables]

# 示例四则运算公式字符串
format_str = "a * b / (c % d) + e - f ** g"
format_str2 = "{abc:efg} + {137a:mn}"

# 提取并打印变量列表
variables_list = extract_variables(format_str2)
print("Extracted variables:", variables_list)

Extracted variables: ['abc:efg', '137a:mn']


In [12]:
import re
def extract_variables(format_str):
    """
    从格式字符串中提取变量名，包括可能包含冒号 ":" 的变量。

    参数:
    format_str: 一个包含变量名的字符串。例如："abc:efg + 137a:mn + name + age"

    返回值:
    一个包含所有提取出的变量名（可能含有冒号）的列表。例如：['abc:efg', '137a:mn', 'name', 'age']。
    """
    # 定义用于匹配可能包含冒号的变量名的正则表达式模式
    variable_pattern = r'\w[\w:()%]*'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(variable_pattern, format_str)
    return variables  # 直接返回找到的变量名列表，无需额外处理

format_str2 = "库存:热卷(板)"

# 提取并打印变量列表
variables_list = extract_variables(format_str2)
print("Extracted variables:", variables_list)

Extracted variables: ['库存:热卷(板)']


In [1]:
import json
import dataworks as dw
dws = dw.DataWorks()

In [2]:
def load_choice_file(file_path):
    """读取choice数据终端导出的数据文件.
        Choiced导出文件格式对应的处理规则. 
        - 原文件第一行为“宏观数据“或类似内容,但read_excel方法未加载该行内容
        - 数据的第一行作为指标标题
        - 第一列作为日期
        - 前六行和最后一行都不是数据内容
        - 剔除“日期”字段为空的行和其他非数据内容（标识数据来源的文字内容）
        
        Choice文件导出注意事项:
        - 导出字段选择:指标名称、频率、单位、来源
        - 日期排序:降序
        - 图形设置:不导出图形
        - 勾选“使用函数方式导出”
    Args:
        file_path (str): Choice导出文件(.xlsx)的绝对路径

    Returns:
        dataframe: 将Choice导出文件内容加载到dataframe,并返回
    """
    if file_path=='':
        return None
    df = pd.read_excel(file_path)
    df.columns = df.iloc[0]
    df.rename(columns={df.columns[0]: 'date'}, inplace=True)
    df = df[6:]
    df.reset_index(drop=True, inplace=True)
    df.dropna(axis=0, subset=['date'], inplace=True)
    df = df[df['date'] != '数据来源：东方财富Choice数据']
    df['date'] = pd.to_datetime(df['date'])
    return df

In [ ]:
common_json = 'setting/common.json'
variety_json = 'setting/variety.json'
symbol_id = "RB"

with open(common_json, encoding='utf-8') as common_file: 
    symbol_dataindex_setting = json.load(common_file)['DataIndex']
with open(variety_json, encoding='utf-8') as variety_file:
    variety_setting = json.load(variety_file)[symbol_id]     
variety_setting['DataIndex'] = {**symbol_dataindex_setting, **variety_setting['DataIndex']} if 'DataIndex' in variety_setting else symbol_dataindex_setting
symbol_setting = variety_setting


In [4]:
import pandas as pd
import re
from asteval import Interpreter

def extract_variables(format_str):
    """从格式字符串中提取变量名"""
    # 正则表达式模式，匹配非空字符（即变量）
    variable_pattern = r'\w[\w:()%]*'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(variable_pattern, format_str)
    return variables  # 直接返回找到的变量名列表，无需额外处理

column_dict= {}
data_frames = []
data_index = symbol_setting['DataIndex']

# dws = gs.dataworks
dws = dw.DataWorks()
for key, value_items in data_index.items():
    # 按照配置文件中的DataFrame键值，将同类内容合并到同一张表中
    df_name = value_items['DataFrame']            
    fields = value_items['Field']
    variables_list = extract_variables(fields)
    if df_name in locals():        
        # 键值是独立字段的，列名修改为key
        if len(variables_list)==1:
            locals()[df_name].rename(columns={variables_list[0]:key}, inplace=True)
    else:
        # 未加载的数据，整表读取到df_name中
        data_source = value_items['Source']
        if data_source=='Choice':
            locals()[df_name] = load_choice_file(value_items['Path'])
        elif data_source=='SQLite':
            locals()[df_name] = dws.get_data_by_symbol(value_items['Path'], symbol_id, '*')
            locals()[df_name]['date'] = pd.to_datetime(locals()[df_name]['date'])
        else:
            continue
        column_dict[df_name] = ['date']
    df = locals()[df_name]            
    if len(variables_list)==1:
        # Field是独立字段的，列名修改为key
        df.rename(columns={variables_list[0]:key}, inplace=True)
        column_dict[df_name].append(key)
    else:                
        # Field是公式表达的，进行解析计算
        aeval = Interpreter()       
        for var in variables_list:                    
            safe_var = re.sub(r'[0-9:]', '', var)
            df.rename(columns={var:safe_var}, inplace=True)                                     
            aeval.symtable[safe_var] = df[safe_var]
        safe_fields = re.sub(r'[0-9:]', '', fields)
        df[key] = aeval.eval(safe_fields)
        column_dict[df_name].append(key)
    # 根据配置中指定的填充方式填充缺失值
    if 'FillNa' in value_items:
        fill_na = value_items['FillNa']
        print(f"symbol: {symbol_id}, key: {key}")
        if fill_na=='Forward':
            df[key] = df[key].ffill()
        elif fill_na=='Backward':
            df[key] = df[key].bfill()
        # elif fill_na=='Interpolate':

symbol: RB, key: 需求
symbol: RB, key: 产量
symbol: RB, key: 开工率
symbol: RB, key: 产能利用率
symbol: RB, key: 库存


/var/folders/hc/6dqjp7bs58s2pcb47s4xfsf80000gn/T/ipykernel_28136/3172956327.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[key] = df[key].ffill()
/var/folders/hc/6dqjp7bs58s2pcb47s4xfsf80000gn/T/ipykernel_28136/3172956327.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[key] = df[key].ffill()


In [5]:
from functools import reduce
for df_key in column_dict:
    df = locals()[df_key]
    df = df[column_dict[df_key]]
    data_frames.append(df)
symbol_data = reduce(lambda left,right: pd.merge(left,right,on='date', how='outer'), data_frames)
# if "利润" in self.symbol_setting:
#     for key, value in enumerate(self.symbol_setting['利润'].items()):
#         variables_list = extract_variables(value)
#         aeval = Interpreter()       
#         for var in variables_list:                                              
#             aeval.symtable[var] = df[var]
#         self.symbol_data[key] = aeval.eval(value)

symbol_data.sort_values(by='date', ascending=True, inplace=True)
columns_to_check = symbol_data.columns[1:]
symbol_data.dropna(subset=columns_to_check, how='all', inplace=True)        

In [34]:
import akshare as ak
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = pd.to_datetime(trade_date)


In [35]:
date_column = 'date'
# 筛选出 date 在 trade_date 中的行
valid_dates_mask = symbol_data[date_column].isin(trade_date)
# 删除不属于交易日的行
new_symbol_data2 = symbol_data.loc[valid_dates_mask]

# 或者，如果您希望直接在原数据框上进行修改（inplace=True）
# symbol_data.drop(symbol_data.index[~valid_dates_mask], inplace=True)

In [ ]:
from datetime import datetime, timedelta
start_date = symbol_data['date'].iloc[0]
end_date = symbol_data['date'].iloc[-1]
interval = timedelta(days=1)  
all_date = [start_date + i * interval for i in range((end_date - start_date).days + 1)]

not_trade_date = sorted(set(all_date) - set(trade_date))

In [7]:
# 重新加载salary计算模块
import importlib
importlib.reload(variety)

<module 'variety' from 'e:\\Document\\Project\\futures_nexus\\variety.py'>

In [8]:
import variety
from variety import *
rb = SymbolData('RB')
rb.merge_data()
# symbol_data = rb._calculate_basis('主力合约收盘价')

symbol: RB, key: 库存
symbol: RB, key: 需求
symbol: RB, key: 产量
symbol: RB, key: 开工率
symbol: RB, key: 产能利用率


e:\Document\Project\futures_nexus\variety.py:210: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  elif fill_na=='Backward':


,date,主力合约收盘价,主力合约结算价,主力合约持仓量,近月合约收盘价,近月合约结算价,近月合约持仓量,现货价格,基差,基差率,仓单,库存,需求,产量,开工率,产能利用率
0,2010-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4568,2024-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1324.80,212.38,218.53,39.02,47.91
4569,2024-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39
4570,2024-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39
4571,2024-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39


In [11]:
field_list = rb.get_data_fields()

In [12]:
data_rank = rb.calculate_data_rank('主力合约',field_list,6)

data_list: ['基差率', '主力合约持仓量', '仓单', '库存', '需求', '产量', '开工率', '产能利用率']


/Volumes/Repository/Projects/futures_nexus/variety.py:357: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_rank = pd.DataFrame()


In [ ]:
signals = rb.get_signals(field_list)

In [1]:
import dash_mantine_components as dmc
from dash import Dash, html, Input, Output, State

layout = html.Div([
    dmc.Stack(
        [
            dmc.Button("Open", id="open-modal"),
            dmc.Button("Close", id="close-modal"),
        ],
        spacing="md",
    )
])

## 并行计算测试

### 使用Numba的并行功能

In [33]:
import numpy as np
import time
from numba import jit, prange

# 定义一个计算大数组元素10次方的函数，使用Numba并行
@jit(nopython=True, parallel=True)
def compute_powers(numbers, power=10):
    result = np.zeros_like(numbers)
    for i in prange(len(numbers)):
        result[i] = numbers[i] ** power
    return result

# 创建一个大数组
size = 100000000
numbers = np.random.rand(size)

start_time = time.time()
powered_numbers = compute_powers(numbers)
end_time = time.time()

print(f"Computation of {size} elements to the power of 10 completed.")
print(f"Execution time with parallel Numba: {end_time - start_time:.6f} seconds")

Computation of 100000000 elements to the power of 10 completed.
Execution time with parallel Numba: 0.417245 seconds


In [27]:
def compute_powers(numbers, power=10):
    result = np.zeros_like(numbers)
    for i in prange(len(numbers)):
        result[i] = numbers[i] ** power
    return result

# 创建一个大数组
size = 100000000
numbers = np.random.rand(size)

start_time = time.time()
powered_numbers = compute_powers(numbers)
end_time = time.time()

print(f"Computation of {size} elements to the power of 10 completed.")
print(f"Execution time with parallel Numba: {end_time - start_time:.6f} seconds")

Computation of 100000000 elements to the power of 10 completed.
Execution time with parallel Numba: 12.351819 seconds
